In [1]:
from  wsj_loader import *
import os
DATA_FOLDER_PATH = '../data'
from tflearn.utils.d

In [2]:
wsj = WSJ()

In [3]:
train, test, dev = wsj.train, wsj.test, wsj.dev

In [4]:
print(train[0].shape, test[0].shape, dev[0].shape)
print(train[1].shape, test[1], dev[1].shape)

(24590,) (268,) (1103,)
(24590,) None (1103,)


In [5]:
def flatten_data_optimizer(data=(), padding=20, freq_band=40, filename="", Limit=0):
    curr_idx = 0
    
    IS_TEST_FILE = True if data[1] is None else False #File has only features no labels
    total_frames = sum([i.shape[0] for i in data[0]])
    num_padded_rows = total_frames + len(data[0])*2*padding # num_frames_utterance + 2*k*num_utterance
    
    X = np.zeros((num_padded_rows, freq_band))
    Y = np.zeros((total_frames))
    Lookup = np.zeros((total_frames))
    padding_frame = np.zeros((padding,freq_band))
    X_idx, Y_idx, LU_idx = 0, 0, 0 #not using LU_idx as Y_idx and LU_idx point to same  
    for idx in range(data[0].shape[0] if Limit == 0 else  Limit):
#         print("Working on Frame number ",idx)
        
        frame = data[0][idx]
        
        X_idx = X_idx + padding_frame.shape[0]
        end_idx = X_idx+frame.shape[0]
        X[ X_idx: end_idx ] = frame
        
        idxes_range = np.array(range(X_idx , end_idx ))
        Lookup[Y_idx: Y_idx + frame.shape[0]] = idxes_range

        X_idx =  X_idx + frame.shape[0] + padding_frame.shape[0]  
        
        
        if IS_TEST_FILE == False:
            labels = data[1][idx]
            Y[Y_idx: Y_idx + frame.shape[0]] = labels
        
        Y_idx = Y_idx + frame.shape[0]
        
    np.save(os.path.join(DATA_FOLDER_PATH, 'flattened', filename + "Flattened_X"), arr=X)
    print("*****************Saved X file **********************")
    print("\tSize : ", X.shape)
    np.save(os.path.join(DATA_FOLDER_PATH, 'flattened', filename + "Flattened_Lookup"), arr=Lookup)
    print("*****************Saved Lookup file **********************")
    print("\tSize : ", Lookup.shape)
    if IS_TEST_FILE == False:
        np.save(os.path.join(DATA_FOLDER_PATH, 'flattened', filename + "Flattened_Y"), arr=Y)
        print("*****************Saved Labels file **********************")
        print("\tSize : ", Y.shape)
#     del X, Lookup, Y
    return X, Lookup, Y

In [6]:
%%time
# del X, L, Y
X, L, Y = flatten_data_optimizer(data=dev, filename='dev')

*****************Saved X file **********************
	Size :  (713414, 40)
*****************Saved Lookup file **********************
	Size :  (669294,)
*****************Saved Labels file **********************
	Size :  (669294,)
Wall time: 1.72 s


In [9]:
%%time
del X, L, Y
X, L, Y = flatten_data_optimizer(data=test, filename='test')

*****************Saved X file **********************
	Size :  (180376, 40)
*****************Saved Lookup file **********************
	Size :  (169656,)
Wall time: 378 ms


In [6]:
%%time
# del X, L, Y
X, L, Y = flatten_data_optimizer(data=train, filename='train')

*****************Saved X file **********************
	Size :  (16432791, 40)
*****************Saved Lookup file **********************
	Size :  (15449191,)
*****************Saved Labels file **********************
	Size :  (15449191,)
Wall time: 42.1 s


In [7]:
def checkr(data, X, Y):
    idx = 0; p =0
    for i in range(data[0].shape[0]):
        idx += 20
        if np.all(X[idx]!=data[0][i][0]):
            print("first entry not mathcing")
        if np.all(X[idx+data[0][i].shape[0]-1]!=data[0][i][-1]):
            print('last entry not matching')
        if np.all(X[idx+5]!=data[0][i][5]):
            print("first entry not mathcing")

        if np.all(Y[p]!=data[1][i][0]):
            print("first label not mathcing")
        if np.all(Y[p+data[0][i].shape[0]-1]!=data[1][i][-1]):
            print('last lavel not matching')
        if np.all(Y[p+5]!=data[1][i][5]):
            print("first label not mathcing")
        p = p + data[0][i].shape[0] 
        idx += data[0][i].shape[0] 
        idx += 20

In [9]:
checkr(train,X,Y)

KeyboardInterrupt: 

NameError: name 'train' is not defined